In [ ]:
import pandas as pd
import numpy as np  

df = pd.read_csv("../data/donnees_fusionnees.csv", index_col="datetime", parse_dates=True)

battery_capacity = 300 # kWh
battery_power_max = 150 # kW
battery_efficiency = 0.9
eta_charge = 0.95
eta_discharge = 0.95
soc = 0 # State Of Charge initial (0%)

df.head()

,power_kW,pv_kW,autoconsommation_kW,injection_kW,soutirage_kW
datetime,,,,,
2023-01-01 00:00:00+00:00,15.893834,0.0,0.0,0.0,15.893834
2023-01-01 01:00:00+00:00,40.392327,0.0,0.0,0.0,40.392327
2023-01-01 02:00:00+00:00,60.323334,0.0,0.0,0.0,60.323334
2023-01-01 03:00:00+00:00,34.624628,0.0,0.0,0.0,34.624628
2023-01-01 04:00:00+00:00,80.772243,0.0,0.0,0.0,80.772243


In [ ]:
soc_list = []
battery_charge = []
battery_discharge = []
import_after_battery = []
export_after_battery = []

for _, row in df.iterrows():
    surplus = row["pv_kW"] - row["power_kW"]

    charge = 0
    discharge = 0
    grid_import = 0
    grid_export = 0

    if surplus > 0:  # Excès de production
        # Charger la batterie
        charge = min(surplus, battery_power_max, (battery_capacity - soc) / eta_charge)
        soc += charge * eta_charge  # Prendre en compte l'efficacité
        grid_export = max(surplus - charge, 0)  # Exporter le surplus restant
    else:  # Déficit de production
        # Décharger la batterie
        deficit = -surplus
        discharge = min(deficit / eta_discharge, battery_power_max, soc)
        soc -= discharge
        supplied = discharge * eta_discharge
        grid_import = max(deficit - supplied, 0)  # Importer le reste du déficit
    
    soc_list.append(soc)
    battery_charge.append(charge)
    battery_discharge.append(discharge)

    import_after_battery.append(grid_import)
    export_after_battery.append(grid_export)


In [ ]:
df["soc"] = soc_list
df["import_after_battery"] = import_after_battery
df["export_after_battery"] = export_after_battery
df["import_grid"] = grid_import
df["export_grid"] = grid_export

In [ ]:
new_import = df["import_after_battery"].sum()
old_import = df["soutirage_kW"].sum()

reduction = (old_import - new_import) / old_import

print(f"Réduction import réseau : {reduction:.2%}")
print("SOC max :", df["soc"].max())
print("SOC final :", df["soc"].iloc[-1])
print("Nombre d'heures où SOC > 0 :", (df["soc"] > 0).sum())

df.head()
df.to_csv("../data/donnees_battery.csv", index = True)

Réduction import réseau : 0.00%
SOC max : 0.0
SOC final : 0.0
Nombre d'heures où SOC > 0 : 0


,power_kW,pv_kW,autoconsommation_kW,injection_kW,soutirage_kW,soc,import_after_battery,export_after_battery,import_grid,export_grid
datetime,,,,,,,,,,
2023-01-01 00:00:00+00:00,15.893834,0.0,0.0,0.0,15.893834,0.0,15.893834,0,66.052665,0
2023-01-01 01:00:00+00:00,40.392327,0.0,0.0,0.0,40.392327,0.0,40.392327,0,66.052665,0
2023-01-01 02:00:00+00:00,60.323334,0.0,0.0,0.0,60.323334,0.0,60.323334,0,66.052665,0
2023-01-01 03:00:00+00:00,34.624628,0.0,0.0,0.0,34.624628,0.0,34.624628,0,66.052665,0
2023-01-01 04:00:00+00:00,80.772243,0.0,0.0,0.0,80.772243,0.0,80.772243,0,66.052665,0


Coût annuel net : 92333.70 €
Coût annuel sans PV : 375348.69 €
Coût annuel sans batterie : 325514.06 €


Économie annuelle : 233180.36 €
Retour sur investissement : 0.2 ans
